# 直近 28日間の Connpass のイベントを取得する

- https://connpass.com/about/api/
- https://connpass.com/robots.txt

In [ ]:
import datetime
import time
import pandas as pd

from IPython.display import display, HTML

import logging

import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

In [ ]:
_FORMAT = '%(asctime)s [%(name)s] [%(levelname)s] %(filename)s %(funcName)s %(message)s'

def get_logger(name, extra=None):
    # インスタンスを作成
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    
    formatter = logging.Formatter(_FORMAT)
    
    # 標準出力に出力
    stdout = logging.StreamHandler()
    stdout.setFormatter(formatter)
    logger.addHandler(stdout)
    return logger

In [ ]:
LOGGER = get_logger("__name__")

In [ ]:
def send(url, header, payload):
    """
    汎用的な webhook 送信用メソッド
    """
    session = requests.Session()
    retries = Retry(total=5,
                    backoff_factor=1,
                    status_forcelist=[429, 500, 502, 503, 504])
    session.mount("https://", HTTPAdapter(max_retries=retries))
    try:
        response = session.get(url=url,
                                headers=header,
                                data=payload.encode("utf-8"),
                                timeout=(10.0, 30.0))
    except requests.exceptions.ConnectTimeout:
        LOGGER.error(f"session timeout: {payload}")
    else:
        # LOGGER.info(f"status code: {response.status_code}")
        # LOGGER.info(f"response: {response.json()}")
        return response

In [ ]:
CONNPASS_URL = "https://connpass.com/api/v1/event/"

In [ ]:
def get_connpass_events(date):
    """
    date に開催されるイベントを取得する
    """
    date_str = date.strftime("%Y%m%d")
    
    start = 1
    events = []
    while True:
        time.sleep(5)
        url = f"{CONNPASS_URL}?ymd={date_str}&start={start}&count=100"
        response = send(url, {}, "")
        response_json = response.json()
        events.extend(response_json["events"])

        if len(events) < response_json["results_available"]:
            start += 100
        else:
            break

        if response_json["results_returned"] == 0:
            break
    return events

### api を使ってイベント取得

In [ ]:
columns = ['event_id', 'title', 'catch', 'description', 'event_url', 'started_at',
    'ended_at', 'limit', 'hash_tag', 'event_type', 'accepted', 'waiting',
    'updated_at', 'owner_id', 'owner_nickname', 'owner_display_name',
    'place', 'address', 'lat', 'lon', 'series']

In [ ]:
# 今日から28日間のイベント取得
today = datetime.date.today()
DAYS = 28

In [ ]:
events_df = pd.DataFrame(columns=columns)
for i in range(DAYS):
    date = today + datetime.timedelta(days = i)
    LOGGER.info(date)
    events = get_connpass_events(date)
    df = pd.DataFrame(events)
    events_df = pd.concat([events_df, df])
events_df.reset_index(inplace=True, drop=True)

In [ ]:
# events_df

### おすすめ検索

In [ ]:
recommend_events_df = events_df.query("event_type == 'participation' and limit > 100 and accepted > 50 and waiting < 10").sort_values("started_at")

In [ ]:
recommend_events_df[['event_id', 'title', 'catch', 'started_at', 'ended_at', 'limit', 'accepted', 'place']]

In [ ]:
for _, row in recommend_events_df.iterrows():
    print(f"{row['event_id']}: {row['event_url']}")
    
    # display(HTML("<hr>"))
    # display(HTML(f"<h3><a href='{row['event_url']}'>{row['title']}</a></h3>"))
    # display(HTML(f"<h1>{row['event_id']}</h1>"))
    # display(HTML(f"<h3>{row['title']}</h1>"))
    # display(HTML(f"<h3>{row['catch']}</h1>"))
    # display(HTML(f"<h3>{row['event_url']}</h1>"))
    # display(HTML(f"<h3>{row['hash_tag']}</h1>"))
    # display(HTML(f"<h3>{row['started_at']}</h1>"))
    # display(HTML(f"<h3>{row['ended_at']}</h1>"))
    # display(HTML(f"<h3>{row['limit']}</h1>"))
    # display(HTML(f"<h3>{row['address']}</h1>"))
    # display(HTML(f"<h3>{row['place']}</h1>"))
    # display(HTML(f"<h3>{row['accepted']}</h1>"))
    # display(HTML(f"<h3>{row['waiting']}</h1>"))
    
    # display(HTML(row["description"]))
    # display(HTML("<hr>"))